<h4>Imports</h4><hr>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers

<h4>Load Dataset && Reshape</h4><hr>

In [ ]:
#from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#x_train(60000, 28, 28) --> x_train(60000, 784)   
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0 #Resimleri flatten dataya çevirdik #-1 belirtilmediği zaman otomatik girilmesini sağlıyor.
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

#numpy array ise tensora çevirmek için
#x_train = tf.convert_to_tensor(x_train) 

<h4>Model Layerlarını Oluşturma</h4><hr>

<li>Yöntem-1 Efektif Olmayan</li>

In [ ]:
#Bu yöntem ile dizi elemanları arka arkaya node olarak eklenir.
model = keras.Sequential([
    keras.Input(shape=(28*28)),                                     #Input Layer
    layers.Dense(512, activation="relu"),                           #Layer-1
    layers.Dense(256, activation="relu"),                           #Layer-2
    layers.Dense(10)                                                #Output Layer
])

"""Yöntem 1 ve 2 de layerlar atandıktan sonra sonra model oluşturulur."""
#arg: outputs = model.layers[-2] | model.get_layer('my_layer').output; 
#model.layers: Tüm layer isimlerini verir.
model = keras.Model(
    inputs=model.inputs,
    outputs=[layer.output for layer in model.layers]
)

<li>Yöntem-2 Yarı Efektif</li>

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(784)))                                 #Input Layer
model.add(layers.Dense(512, activation="relu"))                     #Layer-1
model.add(layers.Dense(256, activation="relu", name="my_layer"))    #Layer-2
model.add(layers.Dense(10))                                         #Output Layer

#Model Özeti Çıkarmak İçin
#print(model.summary())

"""Yöntem 1 ve 2 de layerlar atandıktan sonra sonra model oluşturulur."""
#arg: outputs = model.layers[-2] | model.get_layer('my_layer').output; 
#model.layers: Tüm layer isimlerini verir.
model = keras.Model(
    inputs=model.inputs,
    outputs=[layer.output for layer in model.layers]
)

<li>Yöntem-3 Tercih Edilen</li>

In [ ]:
inputs = keras.Input(shape=(784))                                      #Input Layer
x = layers.Dense(512, activation="relu", name="first_layer")(inputs)   #Layer-1
x = layers.Dense(256, activation="relu", name="second_layer")(x)       #Layer-2
outputs = layers.Dense(10, activation="softmax")(x)                    #Output Layer
#Model Oluşturulur.
model = keras.Model(inputs=inputs, outputs=outputs)



<h4>Compile Model (Loss fonksiyonu ve Backward fonksiyonu ekleme)</h4>

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False), #from_logits=Bu değer sayesinde son çıkıştaki değerleri ilk önce softmax fonksiyonundan geçirmiş oluruz. CategoricalCrossentropy olsaydı ise one-hot encode şeklinde class larımızı verirdik.
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

<h4>Model Eğitimi</h4>

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

<h4>Model Evaluate - Test</h4>

In [ ]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

<h4>Inference(Forward) Model</h4>

In [ ]:
features = model.predict(x_train)
for feature in features:
    print(feature.shape)